# Prioritised experience replay

I've already built a self driving taxi using a concept in machine reinforcement learning called uniform experience replay to aid in the process of deep Q-learning. 

After coming across the paper below however, I've seen the amazing performance gains that this method of a more stratified sampling of replay memory can offer.

Check out the origional  [Google Deepmind 2016 PRIORITIZED EXPERIENCE REPLAY](https://arxiv.org/pdf/1511.05952v3.pdf) if you want a deeper understanding.

## So your taxi drives itself?

You might've gathered by now that this taxi is self driving but what does this mean?

If you're not familiar with what machine learning (ML) is, I recommend you go and check out [my blog post](https://medium.com/@ross.nkama/an-intuitive-understanding-of-machine-learning-6814add2b2a9) to get a high level understanding.

So I've been getting my machine intelligence's Neural Network to use a technique called reinforcement learning to incrementally update it's structure based off of it's ability to find correlations between the state that it's currently experiencing and (from experience) that state's ability to bring about particular rewards and consuquences at particular times.

These experiences I'm talking about are transitions in time where:

$\hspace{1cm} transition = [s,a,s\prime,r]$

Which is to say that a transition is where an action $a$, taken at a state $s$, leads to the agent recieving a reward $r$ (positive or negative) at the next state $s\prime$.

And previously, in my first implementation of the taxi, I stored a bunch of these transition batches into a sliding window of memory and sampled from this memory, a uniform distribution of experiences as is being shown in our first algorithm inspired by [The Effects of Memory Replay in Reinforcement Learning - Stanford university](https://arxiv.org/pdf/1710.06574.pdf):

***
**_Algorithm 1_**: Uniform sampling form replay memory in a DQN
***


1: **INPUT**: minibatch size $M$, capacity $C$, learning rate $\alpha$, discount factor $\gamma$, initial weights $\theta_0$, policy$_{t}$ $\pi$, total transitions $T$.  
2: Initialise replay memory BUFFER with capacity C.  
3: Observe initial state $S_{0}$  
4: for $t = 1$ to $T$ do  
5: $\hspace{0.9cm}$ select hypothetical best action from policy $a_{t}=\pi(S_{t})$  
6: $\hspace{0.9cm}$ observe $r$ and $s\prime$  
7: $\hspace{0.9cm}$ In memory BUFFER, store transition $[s, a, s\prime,r]$  
8: $\hspace{0.9cm}$ From a uniform distribition of BUFFER, sample $[s, a, s\prime,r]$  
9: $\hspace{0.9cm}$  for i = 1 to M do   
10: $\hspace{1.6cm}$ Compute temporal difference error:  
$\hspace{2.8cm}$ $TD_{error} = R(s,a) + \gamma \Sigma_{t=1}^T p(s\prime|_{s,a}) \cdot \overset{max}{_a\prime}(Q(s\prime, a\prime)) - Q(s, a)_{t-1}$  
11: $\hspace{1.65cm}$ Update weights $\theta = \theta + \alpha TD_{error_t}(a,s)$ where $\theta$ directly changes $Q(s, a).$  
12: $\hspace{0.85cm}$ end for  
13: end for
***

Pretty cool right? You might be able to see that without this, we would be learning from experiences with a high temporal relationship to eachother and they would instantly be discarded after they've been used for learning.

However, after reading Google's paper, it'd come to light that this method of sampling could still be improved with prioritised experience replay. I understood that a lot of the useful memories' potential was being wasted as uniform sampling doesn't sample againt the qualitative worth of particular transitions. 

This is really well summarised below:
> "_An RL agent can learn more effectively from some transitions than from others._" - PRIORITIZED EXPERIENCE REPLAY, Google deepmind pg 1

and

> "_Prioritized replay further liberates agents from considering transitions with the same frequency that they are experienced._" - PRIORITIZED EXPERIENCE REPLAY, Google deepmind pg 1

So the paper sheds light to many different methods of prioritised sampling but favours more frequently sampling transitions pertaining to the biggest expected learning value.

## Stochastic temporal difference error prioritised sampling
I really think paper has offered enough evidence, and it really just makes sense that the temporal difference makes a good indicator as to how valuable a learning experience is. If the TD-Error is high at any point in a spacial/temporal field then it means that at that point in the field, our AI's model of reality has bad predictive capability and so it has much to learn from that transition with the high TD-Error thus making that transition more of a priority than others with less TD-Error.

Although I need to be careful... The paper makes clear that if we greedily prioritise samples by:  
$\hspace{0.9cm} a_t = \pi(s \in \overset{max}{\delta}(t \in T))$   
$\hspace{1.5cm}$ where, $\pi$ is the current policy, $\delta =$ TD-Error, $T$ = replay memory and $t$ = transition from replay memory.  

Then we can end up loosing diversity of the transitions that are selected for learning which could get us stuck at local maxima as our agent is unwilling to revisit and explore experiences with lower TD-Errors.

**And this is why we need stochasticity!**

So I guess the challange now is to find a good balance between exploration and exploitation which leads us to our next point.

## Stochastic prioritisation
The paper makes clear that sweeping over the entire replay memory windows to update TD-Errors on each update is expensive and get's more so at a rate of $O(N)$, Which is why we want to update only the TD-Errors which are coupled with transitions that have been selected for replay.

But this only futher exacerbates the greedy prioritisation problem as experiences with a low TD on the first visit will be sunk to the buttom of the prioirity queue and might never be visited again if there's not enough stochasticity in our system as they will be flushed out of the window memory in time.

So here's the stochastic sampling method proposed in the paper to fix the issues above:

$\hspace{2cm}$ $P(i) = \dfrac{P_i^{\alpha}}{\Sigma_{k}P_k^{\alpha}}$

> *This is where where $P(i) > 0$ is the priority of transition $i$ being sampled where the exponent $\alpha$ determines how much prioritization is used, with $\alpha = 0$ corresponding to the uniform case.*

On top of stochasticity, to maintain diversity in the experiences sampled from memory, we need to make sure that edge-case transitions who have a TD-Error near 0 are revisited and this can be done through a bias.

$p_i = |\delta_i| + c$ where $c$ is a small constant which keep's these edge case transition's TD-Errors from being 0, ensuring revisitation.

We could also consider an indirect, rank-based prioritization where:

$\hspace{2cm}$ $P_i = \dfrac{1}{rank(i)}$

In this case we're able to determine $P_i$ because $P_i \propto \dfrac{1}{rank(i)}$ and $rank(i) \propto \delta$ where as $\delta$ of a transition increases, so does $rank(transition)$ causing the priority queue to be sorted by $\delta_i|$ in the binary heap.